In [3]:
%matplotlib qt5

In [4]:
import hyperspy.api as hs

In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import maskselector_gui as msg

ModuleNotFoundError: No module named 'maskselector_gui'

In [ ]:
edx_data = hs.load("C:/Users/Tara/Documents/Research/17-06_TEMCIGS/EDX_fitting/GS_c5/c5_atinterface.rpl")

In [8]:
v= edx_data.to_signal1D()

In [9]:
img = hs.load("C:/Users/Tara/Documents/Research/17-06_TEMCIGS/GS_C5/dm3_Images/ForEDX.dm3")
dim = v.axes_manager.shape
imgBin = img.rebin((dim[0],dim[1]))

In [11]:
#imgBin
msg

NameError: name 'msg' is not defined

In [9]:
v.plot(navigator=imgBin)

In [10]:
# Integrate over the spatial 0 and 1 dimensions to get the average spectra
s = v.integrate1D(0)
s = s.integrate1D(0)

In [12]:
s.plot()

In [ ]:
#s.metadata.Acquisition_instrument.TEM

In [11]:
#azimuth angle between sample holder and detector
#elevation edx detector to horizontal
v.set_signal_type('EDS_TEM')
v.set_microscope_parameters(beam_energy=200, live_time=2008, tilt_stage=15, azimuth_angle=90, elevation_angle=22, energy_resolution_MnKa=131.79)
v.set_elements(['C','O','S','K','Na', 'Cu', 'In', 'Ga', 'Se', 'Fe', 'Zn', 'Cr','Cd','Al', 'Pt'])
v.change_dtype('float32')

In [12]:
v.axes_manager.signal_axes[0].units = 'keV'
v.axes_manager.signal_axes[0].name = 'E'
#check for these values
v.axes_manager.signal_axes[0].scale = 0.01
v.axes_manager.signal_axes[0].offset = -0.1

In [20]:
#len(v.set_elements)

TypeError: object of type 'method' has no len()

In [14]:
#plots each channel, change length for number of elements to be fit
i=0
while i<15:
    v.get_lines_intensity()[i].plot()
    i = i+1

C:\Users\Tara\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [21]:
v.get_lines_intensity()

[<BaseSignal, title: X-ray line intensity of : Al_Ka at 1.49 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : C_Ka at 0.28 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : Cd_La at 3.13 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : Cr_Ka at 5.41 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : Cu_Ka at 8.05 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : Fe_Ka at 6.40 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : Ga_Ka at 9.25 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : In_La at 3.29 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : K_Ka at 3.31 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : Na_Ka at 1.04 keV, dimensions: (128, 128|)>,
 <BaseSignal, title: X-ray line intensity of : O_Ka at 0.52 keV, dimensions: (128,

# Create Mask

In [15]:
mask_obj = msg.Interactive_plot(imgBin.data)

C:\Users\Tara\Anaconda3\lib\site-packages\matplotlib\cbook.py:136: MatplotlibDeprecationWarning: The axisbg attribute was deprecated in version 2.0. Use facecolor instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [19]:
mask_vac = mask_obj.main.mask

In [22]:
mask_vac

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [30]:
np.save('mask.npy', mask_vac)

In [31]:
mask_vac = np.load("mask.npy")

In [32]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
im = ax1.imshow(mask_vac[:,:])

In [35]:
ave_edx_ebic_true = np.zeros_like(s.data[0,0,:], dtype=uint64)
# check that ss can be plotted, make sure it behaves like s, otherwise remove ()
ss = s.deepcopy()
ave_edx_ebic_false = np.zeros_like(s.data[0,0,:], dtype=uint64)
true = 0
false = 0

for i in range(len(v.data[:,0,0])):
    for j in range(len(v.data[0,:,0])):
        if mask_vac[i,j] == True:
            # this is average spectrum inside the polygon
            ss.data[:] = ave_edx_ebic_true[:] + edx.data[i,j,:]
            true = true + 1
        else:
            ave_edx_ebic_false[:] = ave_edx_ebic_false[:] + edx.data[i,j,:]
            false = false + 1

IndexError: too many indices for array

In [36]:
s.data

array([  0.        ,   0.        ,   0.        , ...,  19.16666667,
        26.16666667,  32.16666667])

In [ ]:
x_ev = np.zeros_like(edx.data[0,0,:], dtype=float16)
for i in range(len(x_ev)):
    x_ev[i]=i*9.952-470

### Local averaging

#### Area1

In [ ]:
s_localint1 = np.average(edx_data.data[170:360,180:340,:])

In [ ]:
s_localint1 = np.average(s_localint1[:,:],0)

In [ ]:
x_nm = np.zeros_like(edx_data.data[0,0,:])
for i in range(len(x_nm)):
    x_nm[i]=(i*edx_data.axes_manager[2].scale+edx_data.axes_manager[2].offset)

In [ ]:
fig, ax = plt.subplots(1, 1)
plt.plot(x_nm, s_localint1, lw=2, label='Local ave')
plt.show()

#### Area2

In [ ]:
s_localint2 = np.average(edx_data.data[0:170,0:180,:],0)

In [ ]:
s_localint2 = np.average(s_localint2[:,:],0)

In [ ]:
fig, ax = plt.subplots(1, 1)
plt.plot(x_nm, s_localint2, lw=2, label='Local ave')
plt.show()

#### Area3

In [ ]:
s_localint3= np.average(edx_data.data[460:630,0:180,:],0)

In [ ]:
s_localint3 = np.average(s_localint3[:,:],0)

In [ ]:
fig, ax = plt.subplots(1, 1)
plt.plot(x_nm, s_localint3, lw=2, label='Local ave')
plt.show()